In [ ]:
versioninfo()

In [ ]:
]instantiate

In [ ]:
using DatasetManager, LabDataSources, ParkinsonsDualTaskCoordination, C3D, DSP, DataFrames,
    PlotlyJS, HypothesisTests, Biomechanics, Statistics, Printf, PrettyTables,
    CategoricalArrays, CSV, DataFramesMeta

In [ ]:
rootdir = joinpath(pkgdir(ParkinsonsDualTaskCoordination), "data")

rawpath = joinpath(rootdir, "raw")
genpath = joinpath(rootdir, "generated")

subsets = [
    DataSubset("c3d", Source{C3DFile}, joinpath(rawpath, "c3d"), "S*.c3d"),
    DataSubset("ik", OSimMotion, genpath, "Subject */ik/*.mot.gz"),
    DataSubset("events", V3DEventsSource, genpath, "Subject */events/*.tsv"),
    DataSubset("dflow", Source{RawDFlowPD}, joinpath(rawpath, "dflow"), "S*.txt"),
]

labels = Dict(
    :subject => r"(?<=S)\d+B?",
    :task => r"single|dual"
)
conds = TrialConditions((:subject,:task), labels)

trials = findtrials(subsets, conds)

modelsubset = DataSubset("model", Source{OSimModel}, joinpath(rawpath, "models"), "S*.osim"; dependent=true)
conds = TrialConditions((:model,), Dict(:model => (r".osim$" => "model" => "model")); subject_fmt=r"(?<=S)(?<subject>\d+B?)", required=(:subject,))
findtrials!(trials, [modelsubset], conds)

setfield!.(trials[findall(==("15B")∘subject, trials)], :subject, "15")

demog = CSV.read("../data/demographics.csv", DataFrame)
moreaffected_side = Dict(string.(demog[!,"Participant ID"]) .=> demog[!,"More affected Side"])
addcondition!.(trials, :ma_side => t -> moreaffected_side[subject(t)])

summarize(trials; ignoreconditions=[:ma_side])

## Main analysis

In [ ]:
srs = analyzedataset(trials, OSimMotion) do trial
    analyze(trial; genpath)
end;

## Statistics

In [ ]:
longdf = DatasetManager.stack(srs, [:task,:ma_side])
levels!(longdf.task, ["single", "dual"])
ordered!(longdf.task, true)
sort!(longdf, [:subject,:task])

widedf = unstack(longdf)
gd = groupby(widedf, :task);

In [ ]:
vars = filter(!contains(r"(asym|phase)$"), resultsvariables(srs))
degreevars = vars[findall(contains(r"flex|phase"), vars)]
nondegreevars = setdiff(vars, degreevars);

In [ ]:
CSV.write("../results/results.csv", longdf)

In [ ]:
write_results("../results/results-wide.csv", longdf, [:task])

In [ ]:
sort!(combine(gd, [:numlsteps, :numrsteps] => ((l,r) -> [(mean([l;r]), std([l;r]), minimum([l;r]))]) => [:steps_avg, :steps_std, :steps_min]), :task)

In [ ]:
combine(widedf, :gait_speed => (v -> [(mean(skipmissing(v)), std(skipmissing(v)), minimum(skipmissing(v)))]) => [:avg_gaitvelocity, :gaitvelocity_std, :min_gaitvelocity])

# Statistical Analysis

In [ ]:
function unzip(x::Vector{NTuple{N,T}}) where {N,T}
    out = ntuple(_ -> Vector{T}(undef, 0), N)
    unzip!(out,x)
end

function unzip!(out, x::Vector{NTuple{N,T}}) where {N,T}
    for i in eachindex(x), j in 1:N
        push!(out[j], x[i][j])
    end
    
    return out
end

function dropmissingpairs(x, y)
    a,b = unzip(filter(x -> !any(ismissing, x), zip(x, y) |> collect))
    V = Vector{nonmissingtype(eltype(a))}
    return convert(V, a), convert(V, b)
end

In [ ]:
function CohensDz(test::OneSampleTTest)
    return test.t/sqrt(test.n)
end

function hedges_correction(n1, n2=n1)
    return (1-(3/(4*(n1+n2)-9)))
end

function CohensDₐᵥ(mdiff, sd1, sd2, n1, n2=n1; lessbiased=true)
    correction = lessbiased ? hedges_correction(n1, n2) : 1
    return mdiff/((sd1+sd2)/2)*correction
end

## Original analysis

In [ ]:
function ttest_table(df)
    _df = select(df, r"^(la|ma|hip|shoulder)|task|pci|asym$")
    dt = _df[_df.task .== "dual",:]
    st = _df[_df.task .== "single",:]
    varnames = names(_df, Not([:task,:ma_side]))
    ttests = [[ OneSampleTTest((-)(dropmissingpairs(dt[!, col], st[!, col])...)) for col in intersect(nondegreevars, varnames) ];
        [ OneSampleTTest(circmeand((-)(dropmissingpairs(dt[!, col], st[!, col])...)),
                circstdd((-)(dropmissingpairs(dt[!, col], st[!, col])...)), length(dropmissingpairs(dt[!, col], st[!, col])[1]), 0) for col in intersect(degreevars, varnames) ]]
    variables = [intersect(string.(nondegreevars), varnames); intersect(string.(degreevars), varnames)]
    
    split_and_count(std, x) = (std.(x)..., length(x[1]))
    Gavs = [
        [ CohensDₐᵥ(getproperty(test, :xbar), split_and_count(std, dropmissingpairs(dt[!, var], st[!, var]))...)
            for (test, var) in zip(ttests, intersect(string.(nondegreevars), varnames)) ];
        [ CohensDₐᵥ(getproperty(test, :xbar), split_and_count(circstdd, dropmissingpairs(dt[!, var], st[!, var]))...)
            for (test, var) in zip(ttests, intersect(string.(degreevars), varnames)) ]
        ]
    
    outdf = DataFrame(
        variables = variables,
        ST = map(variables) do var
            if var ∈ degreevars
                return Printf.format(Printf.Format("%.1f ± %.1f"), circmeand(st[!,var] |> skipmissing), circstdd(st[!,var] |> skipmissing))
            else
                return Printf.format(Printf.Format("%.1f ± %.1f"), mean(st[!,var] |> skipmissing), std(st[!,var] |> skipmissing))
            end
        end,
        DT = map(variables) do var
            if var ∈ degreevars
                return Printf.format(Printf.Format("%.1f ± %.1f"), circmeand(dt[!,var] |> skipmissing), circstdd(dt[!,var] |> skipmissing))
            else
                return Printf.format(Printf.Format("%.1f ± %.1f"), mean(dt[!,var] |> skipmissing), std(dt[!,var] |> skipmissing))
            end
        end,
        meandiff = round.(getfield.(ttests, :xbar); sigdigits=3),
        low_ci = round.(first.(confint.(ttests)); digits=1),
        upper_ci = round.(last.(confint.(ttests)); digits=1),
        t = Printf.format.(Ref(Printf.Format("t(%d)=%.2f")), getfield.(ttests, :df), getfield.(ttests, :t)),
        pvalue = clamp.(round.(pvalue.(ttests); digits=3), .001, 1),
        Gav = Printf.format.(Ref(Printf.Format("%.2f")), Gavs),
        CohensDz = Printf.format.(Ref(Printf.Format("%.2f")), CohensDz.(ttests)),
    )

    order = [ "la_shoulder_rom", "ma_shoulder_rom", "la_shoulder_rom_cov", "ma_shoulder_rom_cov",
        "la_shoulder_peak_flex", "ma_shoulder_peak_flex", "la_hip_rom", "ma_hip_rom",
        "la_hip_rom_cov", "ma_hip_rom_cov", "la_hip_peak_flex", "ma_hip_peak_flex", "pci",
        "shoulder_inter_meansd", "hip_inter_meansd", "la_ipsi_meansd", "ma_ipsi_meansd",
        "la_ulimb_intra_meansd", "ma_ulimb_intra_meansd", "la_llimb_intra_meansd",
        "ma_llimb_intra_meansd" ]
    sort!(outdf, :variables; by=(x-> something(findfirst(==(x), order), 100)))
    
    outdf
end

In [ ]:
tt_results = ttest_table(widedf)
pretty_table(tt_results; backend=Val(:html), highlighters=(HTMLHighlighter((d,i,j) -> (j ∈ (8,)) && (d[i,j] ≤ .05), HTMLDecoration(font_weight = "bold")),))

In [ ]:
CSV.write("../results/ttests.csv", tt_results)

## Additional analysis

In [ ]:
st1 = DataFrames.stack(widedf, Not([:subject, :task, :ma_side, :gait_speed]));

In [ ]:
long_split = DataFrames.unstack(@subset(select(st1, Not(:gait_speed)), occursin.(Ref(r"^[ml]a"), :variable) .& .!occursin.(Ref(r"phase$"), :variable)), :task, :value);

In [ ]:
includet("../src/tost.jl")

In [ ]:
CohensDz(x::TwoOneSidedTTest) = CohensDz(x.main)

### Equivalence bounds: standardized effect estimation

In [ ]:
using Distributions

#### Plate et al. (2015)

Using the results of Plate et al. (2015) to estimate unstandardized effects of DT on bilateral differences in DTC for shoulder ROM. 

Plate, A., Sedunko, D., Pelykh, O., Schlick, C., Ilmberger, J. R., & Bötzel, K. (2015). Normative data for arm swing asymmetry: How (a)symmetrical are we? Gait & Posture, 41(1), 13–18. https://doi.org/10.1016/j.gaitpost.2014.07.011

In [ ]:
# Plate 2015

# 3 km/h
ρ = 0.6
st_3 = MvNormal([22.6,19.8], [7.5^2; ρ*(7.5*11.4);; ρ*(7.5*11.4); 11.4^2])

let x = clamp.(rand(st_3, 1000000), 2.5, 70)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    nothing
end

In [ ]:
# 4 km/h
ρ = 0.6
st_4 = MvNormal([26.2,25.0], [7.9^2; ρ*(7.9*12.3);; ρ*(7.9*12.3); 12.3^2])

let x = clamp.(rand(st_4, 1000000), 2.5, 70)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    nothing
end

In [ ]:
# Stroop @ 3.4 km/h
ρ = 0.6
dt1 = MvNormal([26.1,20.0], [9.4^2; ρ*(9.4*11.2);; ρ*(9.4*11.2); 11.2^2])

let x = clamp.(rand(dt1, 1000000), 2.5, 70)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    nothing
end

In [ ]:
# Counting backwards @ 3.4 km/h
ρ = 0.6
dt2 = MvNormal([28.2,26.0], [7.3^2; ρ*(7.3*12.4);; ρ*(7.3*12.4); 12.4^2])

let x = clamp.(rand(dt2, 1000000), 2.5, 70)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    nothing
end

In [ ]:
st_3_vals = clamp.(rand(st_3, 1000000), 2.5, 70)
st_4_vals = clamp.(rand(st_4, 1000000), 2.5, 70)
dt1_vals = clamp.(rand(dt1, 1000000), 2.5, 70)
dt2_vals = clamp.(rand(dt2, 1000000), 2.5, 70);

In [ ]:
for (st, dt) in ((st_3_vals, dt1_vals), (st_3_vals, dt2_vals), (st_4_vals, dt1_vals), (st_4_vals, dt2_vals))
    didtc = map((st, dt) -> (dt[1]-st[1]) - (dt[2]-st[2]), eachcol(st), eachcol(dt))
    @show mean(didtc), std(didtc)
end

In [ ]:
println("Largest raw effect: `4.7°`, Cohen's d: `$(4.7/13)`")

#### Killeen et al. (2018)

Using the results of Killeen et al. (2018) to estimate largest non-pathological unstandardized effects of DT on bilateral differences in DTC for shoulder ROM. Means and SD estimated from paper figures using a [figure digitizer](https://apps.automeris.io/wpd/).

Killeen, T., Elshehabi, M., Filli, L., Hobert, M. A., Hansen, C., Rieger, D., Brockmann, K., Nussbaum, S., Zörner, B., Bolliger, M., Curt, A., Berg, D., & Maetzler, W. (2018). Arm swing asymmetry in overground walking. Scientific Reports, 8(1), 12803. https://doi.org/10.1038/s41598-018-31151-9

In [ ]:
using Random

In [ ]:
dasi = truncated(MixtureModel([Normal(-25,20), Normal(42, 20)], [5/16, 11/16]), -75, 90)
dt_dasi = truncated(MixtureModel([Normal(-25+4.7,20), Normal(42-4.7, 20)], [5/16, 11/16]), -75, 90)

In [ ]:
for i in axes(roms, 2)
   roms[:,i] .= inv(Vag92)(vals[i], rand(ldist))
end

In [ ]:
y .= rand(dasi, 10_000)
PlotlyJS.plot(PlotlyJS.histogram(;x=y, histnorm="probability density"), Layout(;xaxis_range=[-100,100]))

(increase number of samples to get better/smoother histogram if desired)

In [ ]:
# Normal walking
ρ = 0.65
kill_normwalk = MvNormal([32.3,22.8], [18.4^2; ρ*(18.4*12.9);; ρ*(18.4*12.9); 12.9^2])

let x = clamp.(rand(kill_normwalk, 10000), 2.5, 85)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    PlotlyJS.plot(PlotlyJS.histogram(;x=asym, nbinsx=100, histnorm="probability density"), Layout(;xaxis_range=[-100,100]))
    # nothing
end

In [ ]:
# Fast walking
ρ = 0.55
kill_fastwalk = MvNormal([35.7,22.9], [19.3^2; ρ*(19.3*15);; ρ*(19.3*15); 15^2])

let x = clamp.(rand(kill_fastwalk, 10000), 5, 85)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    PlotlyJS.plot(PlotlyJS.histogram(;x=asym, nbinsx=100, histnorm="probability density"), Layout(;xaxis_range=[-100,100]))
    # nothing
end

In [ ]:
# Fast walking
ρ = 0.4
kill_dtwalk = MvNormal([33.3,24.3], [19.6^2; ρ*(19.6*15.1);; ρ*(19.6*15.1); 15.1^2])

let x = clamp.(rand(kill_dtwalk, 10000), 5, 85)
    @show mean(x[1,:]), std(x[1,:])
    @show mean(x[2,:]), std(x[2,:])
    asym = map(x -> (x[1]-x[2])/max(x[1],x[2])*100, eachcol(x))
    @show mean(asym), std(asym)
    PlotlyJS.plot(PlotlyJS.histogram(;x=asym, nbinsx=100, histnorm="probability density"), Layout(;xaxis_range=[-100,100]))
    # nothing
end

In [ ]:
kill_nw_vals = clamp.(rand(kill_normwalk, 1000000), 5, 80)
kill_fw_vals = clamp.(rand(kill_fastwalk, 1000000), 5, 80)
kill_dt_vals = clamp.(rand(kill_dtwalk, 1000000), 5, 80);

In [ ]:
for (st, dt) in ((kill_nw_vals, kill_dt_vals), (kill_fw_vals, kill_dt_vals))
    didtc = map((st, dt) -> (dt[1]-st[1]) - (dt[2]-st[2]), eachcol(st), eachcol(dt))
    @show mean(didtc), std(didtc)
end

#### Riberio et al. (2019)

Using the results of Riberio et al. (2019) to estimate unstandardized effects of bilateral differences in DTC for hip ROM. Although this study was in PD, the dichotomization by left/right, instead of MA/LA sides, should be insensitive to any possible effects of MA/LA side, as there is no population level correlation between more affected side and left/right or dominant/non-dominant dichotomizations.

Ribeiro, T. S., Sousa, A. C. de, Lucena, L. C. de, Santiago, L. M. M., & Lindquist, A. R. R. (2019). Does dual task walking affect gait symmetry in individuals with Parkinson’s disease? European Journal of Physiotherapy, 21(1), 8–14. https://doi.org/10.1080/21679169.2018.1444086

In [ ]:
(38.59-40.89)-(37.15-39.93)

#### Mirelman et al. (2016)

Using the results of Mirelman et al. (2016) to estimate unstandardized effects of DT on bilateral differences in DTC for shoulder ROM CoV.

Byrne, J. E., Stergiou, N., Blanke, D., Houser, J. J., Kurz, M. J., & Hageman, P. A. (2002). Comparison of Gait Patterns between Young and Elderly Women: An Examination of Coordination. Perceptual and Motor Skills, 94(1), 265–280. https://doi.org/10.2466/pms.2002.94.1.265

In [ ]:
(20.47-17.54)*2 # Theoretical largest change if intralimb coordination only changed on one side

#### Byrne et al. (2002)

Using the results of Byrne et al. (2002) to estimate unstandardized effects of DT on bilateral differences in DTC for intralimb coordination.

Byrne, J. E., Stergiou, N., Blanke, D., Houser, J. J., Kurz, M. J., & Hageman, P. A. (2002). Comparison of Gait Patterns between Young and Elderly Women: An Examination of Coordination. Perceptual and Motor Skills, 94(1), 265–280. https://doi.org/10.2466/pms.2002.94.1.265

In [ ]:
pooledvar(sd1, sd2) = √((sd1^2+sd2^2)/2)

In [ ]:
# Difference in elderly women shank-thigh CRP meanSD between normal walking and unilaterally applied ankle weight
(6.58-5.49)/pooledvar(1.51,2.23) # (standardized, eg Cohen's d)

In [ ]:
(6.58-5.49)*2 # Theoretical largest change if intralimb coordination only changed on one side

#### Ghanavati et al. (2014)

Using the results of Ghanavati et al. (2014) to estimate unstandardized effects of DT on bilateral differences in DTC for lower-limb intralimb coordination. Means and SD estimated from Fig. 2A using the earlier linked figure ditizer.

Ghanavati, T., Salavati, M., Karimi, N., Negahban, H., Ebrahimi Takamjani, I., Mehravar, M., & Hessam, M. (2014). Intra-limb coordination while walking is affected by cognitive load and walking speed. Journal of Biomechanics, 47(10), 2300–2305. https://doi.org/10.1016/j.jbiomech.2014.04.038

In [ ]:
ghan_df = DataFrame(
    segment = ["SF_L"; "SF_R"; "TS_L"; "TS_R"; "PT_L"; "PT_R"],
    complex_mean = [3.77, 3.91, 5.81, 6.19, 9.3, 9.6],
    complex_sd = [.18, .15, .23, .27, .59, .59],
    simple_mean = [ 3.87, 4.1, 6.1, 6.4, 10.0, 10.5 ],
    simple_sd = [ .15, .15, .25, .28, .63, .66],
    singletask_mean = [ 4.06, 4.21, 6.31, 6.67, 10.3, 9.95],
    singletask_sd = [ .17, .18, .29, .24, .52, .50])

In [ ]:
unstack(DataFrames.stack(ghan_df, Not(:segment)), :segment, :value)

In [ ]:
df2 = @chain unstack(DataFrames.stack(ghan_df, Not(:segment)), :segment, :value) begin
    @subset(occursin.(r"mean$", :variable))
    @select(:variable, :sf_diff = :SF_L - :SF_R, :ts_diff = :TS_L - :TS_R, :pt_diff = :PT_L - :PT_R)
end

In [ ]:
df2[2,:ts_diff] .- df2[3,:ts_diff]

In [ ]:
df2[2,:pt_diff] .- df2[3,:pt_diff]

In [ ]:
sum(abs2, DataFrames.stack(@subset(ghan_df, occursin.(r"^TS", :segment))[!,[:simple_sd, :singletask_sd]])[!,:value])

In [ ]:
(df2[2,:pt_diff] .- df2[3,:pt_diff])/√(sum(abs2, DataFrames.stack(@subset(ghan_df, occursin.(r"^PT", :segment))[!,[:simple_sd, :singletask_sd]])[!,:value])/4)

Above is a ridiculously large effect

In [ ]:
(df2[2,:ts_diff] .- df2[3,:ts_diff])/√(sum(abs2, DataFrames.stack(@subset(ghan_df, occursin.(r"^TS", :segment))[!,[:simple_sd, :singletask_sd]])[!,:value])/4)

### Tests

In [ ]:
df = @chain long_split begin
    @subset(contains.(:variable, r"^[ml]a(?!_contra)"))
    @transform(:dtc = :dual .- :single)
    @orderby(:subject, :variable)
end;

In [ ]:
ndf = disallowmissing(sort(unstack(df, :subject, :variable, :dtc; renamecols=(x -> string(x, "_dtc"))), :subject); error=false);

In [ ]:
CSV.write("../results/dtc.csv", ndf)

In [ ]:
function tosttest_table(df)
    variables = unique(replace.(names(df, Not(:subject)), r"^[ml]a_" => ""))
    _tosts = Dict(
        "hip_rom_dtc" => TwoOneSidedTTest(df[:, :la_hip_rom_dtc], df[:, :ma_hip_rom_dtc], .5),
        "hip_peak_flex_dtc" => TwoOneSidedTTest(df[:, :la_hip_peak_flex_dtc], df[:, :ma_hip_peak_flex_dtc], .5),
        "hip_rom_cov_dtc" => TwoOneSidedTTest(df[:, :la_hip_rom_cov_dtc], df[:, :ma_hip_rom_cov_dtc], d=.36),
        "shoulder_rom_dtc" => TwoOneSidedTTest(df[:, :la_shoulder_rom_dtc], df[:, :ma_shoulder_rom_dtc], 3.5),
        "shoulder_peak_flex_dtc" => TwoOneSidedTTest(df[:, :la_shoulder_peak_flex_dtc], df[:, :ma_shoulder_peak_flex_dtc], 3.5),
        "shoulder_rom_cov_dtc" => TwoOneSidedTTest(df[:, :la_shoulder_rom_cov_dtc], df[:, :ma_shoulder_rom_cov_dtc], 6),
        "ipsi_meansd_dtc" => TwoOneSidedTTest(dropmissingpairs(df[:, :la_ipsi_meansd_dtc], df[:, :ma_ipsi_meansd_dtc])..., d=.36),
        "ulimb_intra_meansd_dtc" => TwoOneSidedTTest(dropmissingpairs(df[:, :la_ulimb_intra_meansd_dtc], df[:, :ma_ulimb_intra_meansd_dtc])..., d=.36),
        "llimb_intra_meansd_dtc" => TwoOneSidedTTest(dropmissingpairs(df[:, :la_llimb_intra_meansd_dtc], df[:, :ma_llimb_intra_meansd_dtc])..., 0.85),
    )
    tosts = [ _tosts[var] for var in variables ]

    split_and_count(x) = (std.(x)..., length(x[1]))
    Gav = [ CohensDₐᵥ(getproperty(test, :xbar), split_and_count(dropmissingpairs(df[!, "la_"*var], df[!, "ma_"*var]))...) for (test, var) in zip(tosts, variables) ] 
    
    mean_and_std(x) = (mean(x), std(x))
    outdf = DataFrame(
        variables = variables,
        LA = map(variables) do var
            return Printf.format(Printf.Format("%.2g ± %.2g"), mean_and_std(dropmissingpairs(df[!, "la_"*var], df[!, "ma_"*var])[1])...)
        end,
        MA = map(variables) do var
            return Printf.format(Printf.Format("%.2g ± %.2g"), mean_and_std(dropmissingpairs(df[!, "la_"*var], df[!, "ma_"*var])[2])...)
        end,
        meandiff = round.(getproperty.(tosts, :xbar); sigdigits=3),
        low_ci = round.(first.(confint.(tosts)); sigdigits=2),
        upper_ci = round.(last.(confint.(tosts)); sigdigits=2),
        t = Printf.format.(Ref(Printf.Format("t(%d)=%.2f")), getproperty.(tosts, :df), getproperty.(tosts, :t)),
        pvalue = clamp.(round.(pvalue.(tosts); digits=3), .001, 1),
        tost_pvalue = clamp.(round.(tost_pvalue.(tosts); digits=3), .001, 1),
        Gav = Printf.format.(Ref(Printf.Format("%.2f")), Gav),
        CohensDz = Printf.format.(Ref(Printf.Format("%.2f")), CohensDz.(tosts))
    )
 
    order = [ "shoulder_rom_dtc", "shoulder_rom_cov_dtc", "shoulder_peak_flex_dtc", "hip_rom_dtc",
        "hip_rom_cov_dtc", "hip_peak_flex_dtc", "ipsi_meansd_dtc", "ulimb_intra_meansd_dtc",
        "llimb_intra_meansd_dtc" ]
    sort!(outdf, :variables; by=(x-> something(findfirst(==(x), order), 100)))
    
    outdf
end

In [ ]:
tt_results = tosttest_table(ndf)
pretty_table(tt_results; backend=Val(:html), highlighters=(HTMLHighlighter((d,i,j) -> (j ∈ (8,9)) && (d[i,j] ≤ .05), HTMLDecoration(font_weight = "bold")),))